## Workspace API Examples

This notebook demonstrates how to work with Evo Workspaces using **direct API calls**, providing granular control over workspace operations and detailed insight into the underlying service interactions.

### API vs SDK Approach

This notebook uses **direct Workspace API calls** which:
- ✅ Provides full control over API requests and responses
- ✅ Shows detailed API response structures and metadata
- ✅ Allows for advanced customization and fine-tuning
- ✅ Helps understand the underlying service architecture
- ✅ Useful for debugging and advanced use cases

### Want a Simpler, High-Level Interface?

If you prefer a more streamlined experience with less boilerplate code, check out the `sdk-examples.ipynb` notebook in this same directory. The SDK examples show:
- Simplified method calls using `WorkspaceAPIClient`
- Automatic handling of complex API interactions
- Better error handling and validation
- Recommended for most common use cases

In [ ]:
from evo.aio import AioTransport
from evo.common import APIConnector
from evo.oauth import AuthorizationCodeAuthorizer, EvoScopes, OAuthConnector

cache_location = "./notebook-data"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL
org_id = "<your-organization-id>"  # Replace with your organization ID
hub_url = "<your-hub-url>"  # Replace with your Evo hub URL

client_name = "Evo Workspace API Examples"

transport = AioTransport(
    user_agent=client_name,
)

oauth_connector = OAuthConnector(
    transport=transport,
    client_id=client_id,
)

authorizer = AuthorizationCodeAuthorizer(
    oauth_connector=oauth_connector,
    redirect_url=redirect_url,
    scopes=EvoScopes.all_evo | EvoScopes.offline_access,
)

await authorizer.login()

### Create the APIConnector

In [ ]:
# Create the API connector to make direct API calls
connector = APIConnector(hub_url, transport, authorizer)

## Workspace operations

### List all workspaces

List all workspaces in the organization that the current user has access to.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces"

    path_params = {
        "org_id": org_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list workspaces. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    # Display as a table
    workspace_list = [
        {
            "Name": ws["display_name"],
            "ID": ws["id"],
            "User Role": ws["user_role"],
        }
        for ws in response_json["results"]
    ]

    df = pd.DataFrame(workspace_list)
    display(df)
except Exception as e:
    print(f"Error listing workspaces:\n{e}")

### List paginated workspaces

If you have access to more than 50 workspaces, you can use pagination to efficiently iterate through your workspaces.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces"

    path_params = {
        "org_id": org_id,
    }

    query_params = {
        "limit": 5,
        "offset": 0,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        query_params=query_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list workspaces. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    # Display as a table
    workspace_list = [
        {
            "Name": ws["display_name"],
            "ID": ws["id"],
            "User Role": ws["user_role"],
        }
        for ws in response_json["results"]
    ]

    df = pd.DataFrame(workspace_list)
    display(df)
except Exception as e:
    print(f"Error listing workspaces:\n{e}")

### Create a workspace

Create a new workspace with an optional bounding box and coordinate reference system.

In [ ]:
import json
from datetime import datetime
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces"

    path_params = {
        "org_id": org_id,
    }

    body = {
        "name": f"My new workspace {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}",
        "description": "This is a new workspace created via API",
        "bounding_box": {
            "type": "Polygon",
            "coordinates": [
                [
                    [85.8287, -90.0],
                    [85.8287, 69.84278],
                    [-180.0, 69.84278],
                    [-180.0, -90.0],
                    [85.8287, -90.0],
                ]
            ],
        },
        "default_coordinate_system": "EPSG:2134",
    }

    api_response = await connector.call_api(
        method="POST",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        body=body,
        response_types_map={
            "201": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.CREATED:
        raise RuntimeError(f"Error: Failed to create workspace. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Save the workspace ID for later use
    new_workspace_id = response_json["id"]

    # Print the response in a highlighted format
    print(f"Workspace created successfully with ID: {new_workspace_id}")
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    # Display as a table
    workspace_info = {
        "Name": response_json["display_name"],
        "ID": response_json["id"],
        "Description": response_json["description"],
        "User Role": response_json["user_role"],
        "Created At": response_json["created_at"],
    }

    df = pd.DataFrame([workspace_info]).T
    df.columns = ["Workspace info"]
    display(df)
except Exception as e:
    print(f"Error creating workspace:\n{e}")

### Fetch details for a specific workspace

Retrieve the details of a specific workspace by its ID.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces/{workspace_id}"

    path_params = {
        "org_id": org_id,
        "workspace_id": new_workspace_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to get workspace. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    # Display as a table
    workspace_info = {
        "Name": response_json["display_name"],
        "ID": response_json["id"],
        "User Role": response_json["user_role"],
    }

    df = pd.DataFrame([workspace_info]).T
    df.columns = ["Workspace info"]
    display(df)
except Exception as e:
    print(f"Error fetching workspace:\n{e}")

### Update a workspace

Update the name and description of an existing workspace.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces/{workspace_id}"

    path_params = {
        "org_id": org_id,
        "workspace_id": new_workspace_id,
    }

    body = {
        "name": "My updated workspace",
        "description": "This workspace was updated via API",
    }

    api_response = await connector.call_api(
        method="PATCH",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        body=body,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to update workspace. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    # Display as a table
    workspace_info = {
        "Name": response_json["display_name"],
        "ID": response_json["id"],
        "Description": response_json["description"],
        "User Role": response_json["user_role"],
        "Created At": response_json["created_at"],
    }

    df = pd.DataFrame([workspace_info]).T
    df.columns = ["Workspace info"]
    display(df)
except Exception as e:
    print(f"Error updating workspace:\n{e}")

### Delete a workspace

When you delete a workspace, it will be soft-deleted. Deleted workspaces will no longer be accessible through list workspaces by default.

In [ ]:
from http import HTTPStatus

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces/{workspace_id}"

    path_params = {
        "org_id": org_id,
        "workspace_id": new_workspace_id,
    }

    api_response = await connector.call_api(
        method="DELETE",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "204": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.NO_CONTENT:
        raise RuntimeError(f"Error: Failed to delete workspace. Status: {status}")

    print(f"Successfully deleted workspace with ID: {new_workspace_id}")
except Exception as e:
    print(f"Error deleting workspace:\n{e}")

### Fetch details of a deleted workspace

You must include the `deleted=true` query parameter to fetch details of a deleted workspace.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces/{workspace_id}"

    path_params = {
        "org_id": org_id,
        "workspace_id": new_workspace_id,
    }

    query_params = {
        "deleted": "true",
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        query_params=query_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to get deleted workspace. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    # Display as a table
    workspace_info = {
        "Name": response_json["display_name"],
        "ID": response_json["id"],
        "User Role": response_json["user_role"],
    }

    df = pd.DataFrame([workspace_info]).T
    df.columns = ["Workspace info"]
    display(df)
except Exception as e:
    print(f"Error fetching deleted workspace:\n{e}")

### Restore a deleted workspace

In [ ]:
import json
from http import HTTPStatus

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces/{workspace_id}"

    path_params = {
        "org_id": org_id,
        "workspace_id": new_workspace_id,
    }

    body = {
        "deleted": False,
    }

    api_response = await connector.call_api(
        method="POST",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        body=body,
        response_types_map={
            "204": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.NO_CONTENT:
        raise RuntimeError(f"Error: Failed to restore workspace. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    print(f"Successfully restored workspace with ID: {new_workspace_id}")
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))
except Exception as e:
    print(f"Error restoring workspace:\n{e}")

## User operations

Users can be added to a workspace with one of three user roles:
- Owner
- Editor
- Viewer

Workspace roles follow these rules:
- Users who have the owner role in a workspace can add users with any one of the three roles.
- Users who have the editor role in a workspace can add users with the either the editor or the viewer role.
- Users who have the viewer role can't add other users to a workspace.

### Fetch details of the current user's workspace role

Get the role of the currently authenticated user in a specific workspace.

In [ ]:
import json
from http import HTTPStatus

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces/{workspace_id}/current-user-role"

    path_params = {
        "org_id": org_id,
        "workspace_id": new_workspace_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to get current user role. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    print(f"\nCurrent user's role in workspace: {response_json['role']}")
except Exception as e:
    print(f"Error fetching current user role:\n{e}")

### List users with access to a workspace

List all users who have access to a specific workspace and their roles.

In [ ]:
import json
from http import HTTPStatus

import pandas as pd
from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces/{workspace_id}/users"

    path_params = {
        "org_id": org_id,
        "workspace_id": new_workspace_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.OK:
        raise RuntimeError(f"Error: Failed to list users. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    # Display as a table
    user_roles_list = [
        {
            "Full name": user["full_name"],
            "Email": user["email"],
            "Role": user["role"],
            "User ID": user["user_id"],
        }
        for user in response_json["results"]
    ]

    df = pd.DataFrame(user_roles_list)
    display(df)
except Exception as e:
    print(f"Error listing users:\n{e}")

### Assign a user to a workspace

Add a user to a workspace with a specific role (owner, editor, or viewer).

In [ ]:
import json
from http import HTTPStatus

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces/{workspace_id}/users"

    # Replace with the actual user ID you want to assign
    user_id_to_assign = "<some-user-id>"
    email_to_assign = "<some-user-email>"

    path_params = {
        "org_id": org_id,
        "workspace_id": new_workspace_id,
    }

    body = {
        "user_id": user_id_to_assign,
        "email": email_to_assign,
        "role": "viewer",  # Can be "owner", "editor", or "viewer"
    }

    api_response = await connector.call_api(
        method="POST",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        body=body,
        response_types_map={
            "201": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.CREATED:
        raise RuntimeError(f"Error: Failed to assign user role. Status: {status}")

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))
except Exception as e:
    print(f"Error assigning user role:\n{e}")

### Remove a user from a workspace

Remove a user's access to a workspace.

In [ ]:
from http import HTTPStatus

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/workspaces/{workspace_id}/users/{user_id}"

    # Replace with the actual user ID you want to remove
    user_id_to_remove = "<some-user-id>"

    path_params = {
        "org_id": org_id,
        "workspace_id": new_workspace_id,
        "user_id": user_id_to_remove,
    }

    api_response = await connector.call_api(
        method="DELETE",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "204": HTTPResponse,
        },
    )

    status = api_response.status
    if status != HTTPStatus.NO_CONTENT:
        raise RuntimeError(f"Error: Failed to remove user. Status: {status}")

    print(f"Successfully removed user {user_id_to_remove} from workspace")
except Exception as e:
    print(f"Error removing user:\n{e}")